# Tech Challenge - Fase 3: Previsão de Ações GOOG com Machine Learning

**Aluno:** Mateus Bressan
**Curso:** Pós-Graduação em Machine Learning Engineering

## Introdução

Bem-vindos à apresentação do Tech Challenge da Fase 3! O objetivo deste projeto é desenvolver uma solução completa de Machine Learning para prever o preço de fechamento das ações da Google (GOOG). 

O desafio envolveu as seguintes etapas principais, conforme solicitado: 
1.  **Coleta e Armazenamento de Dados:** Obter dados históricos e "em tempo real" das ações e armazená-los em um banco de dados.
2.  **Modelagem de Machine Learning:** Treinar um modelo para prever o preço das ações. 
3.  **API e Dashboard:** Criar uma API para servir as previsões e um dashboard para visualização. 
4.  **Documentação:** Manter o código documentado e versionado no GitHub.

Vamos detalhar cada uma dessas etapas!

In [2]:
# Instalação das principais dependências
!pip install yfinance pandas numpy scikit-learn matplotlib sqlalchemy psycopg2-binary python-dotenv flask joblib --quiet

print("Bibliotecas instaladas!")

# Importações principais utilizadas no projeto
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sqlalchemy import create_engine, Column, Integer, Float, Date, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv
import os
import base64
from io import BytesIO
from datetime import datetime, timedelta
import pickle


print("Bibliotecas importadas!")

Bibliotecas instaladas!
Bibliotecas importadas!



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Coleta de Dados

A primeira etapa crucial é a coleta de dados. Utilizamos a biblioteca `yfinance` para buscar dados históricos e do dia (simulando tempo real) das ações da GOOG. [cite: main.py]

Criamos uma classe `ColetorDeDados` para organizar essa funcionalidade.

In [ ]:
# Definição da Classe ColetorDeDados (igual ao Colab)
class ColetorDeDados:
    """Classe para coleta de dados históricos e 'em tempo real' de ações."""

    def __init__(self, ticker, data_inicio=None, data_fim=None):
        self.ticker = ticker
        self.data_inicio = data_inicio
        self.data_fim = data_fim if data_fim else datetime.now().strftime("%Y-%m-%d")

    def coletar_dados_historicos(self):
        """Coleta dados históricos de ações."""
        print(f"Coletando dados históricos para {self.ticker} de {self.data_inicio} até {self.data_fim}")
        dados = yf.download(self.ticker, start=self.data_inicio, end=self.data_fim)
        if dados.empty:
            print(f"Nenhum dado histórico encontrado para {self.ticker} no período.")
            return pd.DataFrame()
        dados.reset_index(inplace=True)
        # Renomeando colunas para o padrão do projeto
        dados.rename(columns={'Date': 'data_pregao', 'Open': 'abertura', 'High': 'alta', 'Low': 'baixa', 'Close': 'fechamento', 'Volume': 'volume'}, inplace=True)
        # Selecionando e ordenando colunas relevantes
        dados = dados[['data_pregao', 'abertura', 'alta', 'baixa', 'fechamento', 'volume']]
        # Converter a coluna de data para apenas data (sem hora) se necessário
        dados['data_pregao'] = pd.to_datetime(dados['data_pregao']).dt.date
        print(f"Dados históricos coletados: {dados.shape[0]} registros.")
        return dados

    def coletar_dados_tempo_real(self):
        """Coleta dados intraday (intervalo de 1 minuto) do dia atual."""
        print(f"Coletando dados 'em tempo real' (intraday) para {self.ticker}")
        # Usamos period='1d' e interval='1m' para pegar os dados mais recentes do dia
        dados = yf.download(self.ticker, period='1d', interval='1m')
        if not dados.empty:
            # Renomeando colunas
            dados.rename(columns={'Open': 'abertura', 'High': 'alta', 'Low': 'baixa', 'Close': 'fechamento', 'Volume': 'volume'}, inplace=True)
            dados = dados[['abertura', 'alta', 'baixa', 'fechamento', 'volume']]
            dados.reset_index(inplace=True)
            dados.rename(columns={'Datetime': 'data'}, inplace=True)
             # Converter a coluna de data para datetime
            dados['data'] = pd.to_datetime(dados['data'])
            print(f"Dados 'em tempo real' coletados: {dados.shape[0]} registros.")
        else:
            print("Não foram encontrados dados 'em tempo real' no momento (mercado pode estar fechado).")
        return dados

# Exemplo de uso da coleta de dados históricos
ticker = "GOOG"
data_inicio_historico = "2023-01-01" # Ajuste a data inicial se desejar
data_fim_historico = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d") # Até ontem

coletor_hist = ColetorDeDados(ticker, data_inicio_historico, data_fim_historico)
dados_historicos = coletor_hist.coletar_dados_historicos()

if not dados_historicos.empty:
    print("\nPrimeiros 5 registros históricos:")
    print(dados_historicos.head())
else:
    print("\nNão foi possível coletar dados históricos.")

# Exemplo de uso da coleta de dados "em tempo real"
coletor_rt = ColetorDeDados(ticker)
dados_rt = coletor_rt.coletar_dados_tempo_real()

if not dados_rt.empty:
    print("\nÚltimos 5 registros 'em tempo real' (se disponíveis):")
    print(dados_rt.tail())
else:
    print("\nNão há dados 'em tempo real' disponíveis.")

Coletando dados históricos para GOOG de 2022-01-01 até 2025-04-03
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Dados históricos coletados: 815 registros.

Primeiros 5 registros históricos:
Price  data_pregao    abertura        alta       baixa  fechamento    volume
Ticker                    GOOG        GOOG        GOOG        GOOG      GOOG
0       2022-01-03  143.794404  144.863847  142.825999  144.390579  25214000
1       2022-01-04  144.864359  145.918859  143.138176  143.735703  22928000
2       2022-01-05  143.501302  143.617755  136.875186  137.004578  49642000
3       2022-01-06  136.849312  139.027502  136.118779  136.902557  29050000
4       2022-01-07  137.254895  137.602993  135.148873  136.358643  19408000
Coletando dados 'em tempo real' (intraday) para GOOG


[*********************100%***********************]  1 of 1 completed

Dados 'em tempo real' coletados: 390 registros.

Últimos 5 registros 'em tempo real' (se disponíveis):
Price                       data    abertura        alta       baixa  \
Ticker                                  GOOG        GOOG        GOOG   
385    2025-04-04 19:55:00+00:00  148.550003  148.570007  147.699997   
386    2025-04-04 19:56:00+00:00  147.880005  148.160004  147.740005   
387    2025-04-04 19:57:00+00:00  147.860001  147.860001  147.550003   
388    2025-04-04 19:58:00+00:00  147.649994  147.839996  147.649994   
389    2025-04-04 19:59:00+00:00  147.729996  147.839996  147.639999   

Price   fechamento  volume  
Ticker        GOOG    GOOG  
385     147.889999  543889  
386     147.850006  288737  
387     147.639999  283057  
388     147.729996  378320  
389     147.639999  898314  


## 3. Armazenamento de Dados

Os dados coletados precisam ser armazenados de forma persistente. Para isso, escolhemos um banco de dados PostgreSQL e utilizamos SQLAlchemy como ORM (Object-Relational Mapper) para facilitar a interação entre o Python e o banco. [cite: main.py, 6]

Definimos um modelo (`HistoricoAcao`) que representa a tabela no banco e funções para inserir e consultar os dados.

*Observação: A execução das funções de banco de dados abaixo requer um servidor PostgreSQL configurado e acessível, com as credenciais definidas em um arquivo `.env` ou variáveis de ambiente.*

In [1]:
# --- Código do Banco de Dados (SQLAlchemy) ---
# Este código TENTA se conectar ao banco de dados PostgreSQL local.
# Certifique-se de que o servidor está rodando e o .env está configurado.

# Configuração do SQLAlchemy
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
engine = None
Session = None
Base = declarative_base()
db_connection_ok = False

try:
    print(f"Tentando conectar ao banco: {DATABASE_URL.replace(os.getenv('DB_PASSWORD', '****'), '****')}") # Esconde a senha no print
    engine = create_engine(DATABASE_URL)
    # Testa a conexão
    with engine.connect() as connection:
         print("Conexão com o banco de dados estabelecida com sucesso!")
         db_connection_ok = True
    Session = sessionmaker(bind=engine)
except Exception as e:
    print(f"ERRO: Não foi possível conectar ao banco de dados.")
    print(f"Detalhe do erro: {e}")
    print("Verifique se o servidor PostgreSQL está rodando e se as credenciais no arquivo .env estão corretas.")
    print("As operações de banco de dados serão puladas.")

# Modelo da tabela (igual ao original)
class HistoricoAcao(Base):
    __tablename__ = 'historico_acao'
    id = Column(Integer, primary_key=True, autoincrement=True)
    data_pregao = Column(Date, nullable=False)
    abertura = Column(Float)
    alta = Column(Float)
    baixa = Column(Float)
    fechamento = Column(Float)
    volume = Column(Integer)

# Função para criar a tabela se não existir
def criar_tabela_se_nao_existir():
    if engine:
        try:
            print(f"Verificando/Criando tabela '{HistoricoAcao.__tablename__}'...")
            Base.metadata.create_all(engine)
            print("Tabela verificada/criada.")
        except Exception as e:
            print(f"Erro ao verificar/criar tabela: {e}")

# Função para armazenar dados (usando a conexão real)
def armazenar_dados_rds(dados, session_factory):
    if not session_factory or dados.empty:
        print("Armazenamento pulado (sem sessão de DB ou sem dados).")
        return

    print(f"\n--- Armazenando dados na tabela '{HistoricoAcao.__tablename__}' ---")
    session = session_factory()
    try:
        # Exclui os dados antigos (conforme main.py)
        print("Excluindo dados antigos...")
        num_deleted = session.query(HistoricoAcao).delete()
        session.commit()
        print(f"{num_deleted} registros antigos excluídos.")

        # Insere os novos dados
        print(f"Inserindo {len(dados)} novos registros...")
        registros_obj = []
        for _, row in dados.iterrows():
             # Certifica que a data está no formato date, não datetime
            data_corrigida = row['data_pregao']
            if isinstance(data_corrigida, datetime):
                data_corrigida = data_corrigida.date()

            registro = HistoricoAcao(
                data_pregao=data_corrigida,
                abertura=row['abertura'],
                alta=row['alta'],
                baixa=row['baixa'],
                fechamento=row['fechamento'],
                volume=int(row['volume']) if pd.notna(row['volume']) else None # Trata volume NaN
            )
            registros_obj.append(registro)

        session.add_all(registros_obj)
        session.commit()
        print("Dados armazenados com sucesso no banco de dados.")

    except Exception as e:
        print(f"Erro ao armazenar dados no RDS: {e}")
        session.rollback()
    finally:
        session.close()
    print("--- Fim do armazenamento ---")


# Função para ler dados (usando a conexão real)
def ler_dados_rds(session_factory):
    if not session_factory:
        print("Leitura pulada (sem sessão de DB). Retornando DataFrame vazio.")
        return pd.DataFrame()

    print(f"\n--- Lendo dados da tabela '{HistoricoAcao.__tablename__}' ---")
    session = session_factory()
    try:
        query = session.query(HistoricoAcao).order_by(HistoricoAcao.data_pregao)
        dados_lidos = pd.read_sql(query.statement, engine)
        print(f"{len(dados_lidos)} registros lidos do banco de dados.")
        return dados_lidos
    except Exception as e:
        print(f"Erro ao ler dados do RDS: {e}")
        return pd.DataFrame() # Retorna DataFrame vazio em caso de erro
    finally:
        session.close()
    print("--- Fim da leitura ---")

# Fluxo Principal do Banco de Dados (só executa se a conexão foi ok)
dados_para_analise = pd.DataFrame() # Inicializa vazio

if db_connection_ok:
    criar_tabela_se_nao_existir()
    if not dados_historicos.empty:
        armazenar_dados_rds(dados_historicos, Session)
        dados_para_analise = ler_dados_rds(Session)
    else:
         print("Não há dados históricos para armazenar ou ler do banco.")
         # Tenta ler o que já existe no banco caso a coleta falhe mas o banco funcione
         print("Tentando ler dados existentes no banco...")
         dados_para_analise = ler_dados_rds(Session)

else:
    print("\nUsando dados históricos em memória (coletados anteriormente) pois não há conexão com o DB.")
    # Usa os dados coletados se a conexão com o DB falhar
    dados_para_analise = dados_historicos.copy()


if not dados_para_analise.empty:
    print("\nDados carregados para análise (do Banco de Dados ou Memória):")
    print(dados_para_analise.head())
    print(f"\nTotal de registros para análise: {len(dados_para_analise)}")
    # Garante que a coluna de data seja datetime para plotagem
    dados_para_analise['data_pregao'] = pd.to_datetime(dados_para_analise['data_pregao'])
else:
    print("\nERRO: Não foi possível obter dados para análise (nem do DB, nem da coleta). Verifique os passos anteriores.")

NameError: name 'os' is not defined

## 4. Exploração e Pré-processamento de Dados

Antes de treinar o modelo, é essencial explorar e pré-processar os dados lidos do banco (ou da nossa simulação aqui).

**Etapas:**
1.  **Análise Exploratória:** Visualizar a série temporal do preço de fechamento. [cite: main.py]
2.  **Engenharia de Features:** Criar novas features que podem ajudar o modelo, como:
    * Média Móvel (7 dias): Suaviza a série e indica tendência.
    * Retorno Diário (%): Variação percentual do preço de fechamento. [cite: main.py]
3.  **Limpeza:** Remover valores ausentes (`NaN`) que surgem após o cálculo da média móvel e retorno diário. [cite: main.py]
4.  **Normalização:** Colocar as features numéricas na mesma escala (0 a 1) usando `MinMaxScaler`. Isso é importante para muitos algoritmos de ML. [cite: main.py]
5.  **Divisão Treino/Teste:** Separar os dados em conjuntos de treino e teste para avaliar o modelo de forma justa. Usamos `shuffle=False` para manter a ordem temporal. [cite: main.py]

In [ ]:
# Usando os dados carregados na célula anterior (do DB ou memória)
if not dados_para_analise.empty:
    dados_processamento = dados_para_analise.copy()

    # 1. Análise Exploratória (Visualização)
    print("\n--- Exploração de Dados ---")
    # Garante que volume seja numérico, tratando erros
    dados_processamento['volume'] = pd.to_numeric(dados_processamento['volume'], errors='coerce')
    print(dados_processamento.describe())

    plt.figure(figsize=(14, 5))
    plt.plot(dados_processamento['data_pregao'], dados_processamento['fechamento'], label='Preço de Fechamento')
    plt.title(f'Preço de Fechamento {ticker}')
    plt.xlabel('Data')
    plt.ylabel('Preço (USD)')
    plt.legend()
    plt.grid(True)
    plt.show()

    # 2. Engenharia de Features
    print("\n--- Engenharia de Features ---")
    dados_processamento['media_movel'] = dados_processamento['fechamento'].rolling(window=7).mean()
    dados_processamento['retorno_diario'] = dados_processamento['fechamento'].pct_change()
    print("Features 'media_movel' e 'retorno_diario' criadas.")

    # 3. Limpeza (Remover NaNs)
    print("\n--- Limpeza de Dados ---")
    print(f"Registros antes do dropna: {len(dados_processamento)}")
    dados_processamento.dropna(inplace=True)
    print(f"Registros após o dropna: {len(dados_processamento)}")

    if not dados_processamento.empty:
        # 4. Normalização
        print("\n--- Normalização (MinMaxScaler) ---")
        scaler = MinMaxScaler()
        features_para_escalar = ['fechamento', 'volume', 'media_movel', 'retorno_diario']
        # Verifica se todas as colunas existem antes de escalar
        cols_existentes = [col for col in features_para_escalar if col in dados_processamento.columns]
        if len(cols_existentes) == len(features_para_escalar):
            dados_processamento[features_para_escalar] = scaler.fit_transform(dados_processamento[features_para_escalar])
            print("Features escaladas:")
            print(dados_processamento[features_para_escalar].head())

            # Salva o scaler para uso posterior
            scaler_filename = 'scaler.pkl'
            try:
                with open(scaler_filename, 'wb') as f:
                    pickle.dump(scaler, f)
                print(f"Scaler salvo como '{scaler_filename}'")
            except Exception as e:
                 print(f"Erro ao salvar scaler: {e}")


            # 5. Divisão Treino/Teste
            print("\n--- Divisão Treino/Teste ---")
            features = features_para_escalar # Usa as mesmas features escaladas
            target = 'fechamento' # Queremos prever o fechamento

            X = dados_processamento[features]
            y = dados_processamento[target]

            # Dividindo os dados (80% treino, 20% teste), sem embaralhar
            X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, shuffle=False)

            print(f"Tamanho do conjunto de Treino (X): {X_treino.shape}")
            print(f"Tamanho do conjunto de Teste (X): {X_teste.shape}")
        else:
            print(f"ERRO: Nem todas as colunas {features_para_escalar} encontradas para escalar.")
    else:
        print("ERRO: Não há dados após a limpeza para continuar o pré-processamento.")

else:
    print("\nERRO: Sem dados para processar. Verifique a coleta e a conexão com o banco de dados.")

## 5. Treinamento do Modelo de Machine Learning

Com os dados preparados, podemos treinar nosso modelo. Escolhemos o **Random Forest Regressor**, um modelo de ensemble robusto e eficaz para tarefas de regressão, como a previsão de preços. [cite: main.py, techchallenge3/README.md]

Para encontrar os melhores hiperparâmetros (configurações) do modelo, utilizamos o `GridSearchCV`, que testa várias combinações e seleciona a melhor com base na validação cruzada. [cite: main.py]

In [ ]:
# Verifica se X_treino e y_treino existem e não estão vazios
if 'X_treino' in locals() and 'y_treino' in locals() and not X_treino.empty:
    print("\n--- Treinamento do Modelo (Random Forest Regressor) ---")

    # Definição da grade de hiperparâmetros (pode usar a original do main.py)
    param_grid = {
        'n_estimators': [100, 200, 300], # Conforme main.py
        'max_depth': [5, 10, 15],         # Conforme main.py
        'min_samples_split': [2, 5, 10],  # Conforme main.py
        'min_samples_leaf': [1, 2, 4]     # Conforme main.py
    }
    # param_grid_reduzida = { # Use esta se quiser teste mais rápido
    #    'n_estimators': [100, 150],
    #    'max_depth': [5, 10],
    #    'min_samples_split': [5, 10],
    #    'min_samples_leaf': [2, 4]
    # }


    # Inicializa o modelo
    modelo_rf = RandomForestRegressor(random_state=42, n_jobs=-1)

    # Configura o GridSearchCV
    grid_search = GridSearchCV(estimator=modelo_rf, param_grid=param_grid, cv=3,
                               scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

    # Treina o modelo usando GridSearchCV
    print("Iniciando GridSearchCV...")
    grid_search.fit(X_treino, y_treino)

    # Obtém o melhor modelo
    melhor_modelo = grid_search.best_estimator_

    print(f"\nMelhores hiperparâmetros encontrados: {grid_search.best_params_}")
    print("Modelo treinado com sucesso!")

    # Salva o modelo treinado localmente
    modelo_filename = 'modelo.pkl'
    try:
        with open(modelo_filename, 'wb') as f:
            pickle.dump(melhor_modelo, f)
        print(f"Modelo treinado salvo como '{modelo_filename}'")
    except Exception as e:
        print(f"Erro ao salvar o modelo: {e}")

else:
    print("\nERRO: Conjuntos de treino (X_treino, y_treino) não disponíveis ou vazios. Treinamento pulado.")

## 6. Avaliação do Modelo

Após o treinamento, precisamos avaliar o desempenho do modelo no conjunto de teste (dados que ele nunca viu antes). Usamos as métricas:

* **Mean Squared Error (MSE):** Média dos erros quadrados. Penaliza mais os erros grandes.
* **Mean Absolute Error (MAE):** Média dos erros absolutos. Mais interpretável na unidade original do alvo (após reverter a escala). [cite: main.py]

Também visualizamos as previsões do modelo em comparação com os valores reais no conjunto de teste.

In [ ]:
# Verifica se o modelo foi treinado e os dados de teste existem
if 'melhor_modelo' in locals() and 'X_teste' in locals() and not X_teste.empty:
    print("\n--- Avaliação do Modelo ---")

    # Carrega o scaler salvo anteriormente
    scaler_carregado = None
    scaler_filename = 'scaler.pkl'
    try:
        with open(scaler_filename, 'rb') as f:
            scaler_carregado = pickle.load(f)
        print(f"Scaler '{scaler_filename}' carregado para avaliação.")
    except Exception as e:
        print(f"Erro ao carregar scaler '{scaler_filename}': {e}. A avaliação desnormalizada pode falhar.")


    # Fazer previsões no conjunto de teste
    previsoes_teste_scaled = melhor_modelo.predict(X_teste)

    # Calcular métricas com dados escalados
    mse_scaled = mean_squared_error(y_teste, previsoes_teste_scaled)
    mae_scaled = mean_absolute_error(y_teste, previsoes_teste_scaled)
    print(f"\nMSE (escalado): {mse_scaled:.6f}")
    print(f"MAE (escalado): {mae_scaled:.6f}")

    # Reverter a escala para interpretação (se o scaler foi carregado)
    if scaler_carregado:
        try:
            # Cria cópias para evitar SettingWithCopyWarning
            X_teste_copy = X_teste.copy()
            X_teste_copy['target_pred_scaled'] = previsoes_teste_scaled
            X_teste_copy['target_real_scaled'] = y_teste

            # Recria o array completo para inverse_transform
            # Previsões - usa valores reais das outras features + previsão da target
            temp_pred_array = X_teste.copy()
            temp_pred_array['fechamento'] = previsoes_teste_scaled
            previsoes_teste_unscaled = scaler_carregado.inverse_transform(temp_pred_array)[:, features.index(target)]


            # Reais - usa valores reais das outras features + valor real da target
            temp_real_array = X_teste.copy()
            temp_real_array['fechamento'] = y_teste
            y_teste_unscaled = scaler_carregado.inverse_transform(temp_real_array)[:, features.index(target)]


            # Calcular métricas com dados na escala original
            mse_unscaled = mean_squared_error(y_teste_unscaled, previsoes_teste_unscaled)
            mae_unscaled = mean_absolute_error(y_teste_unscaled, previsoes_teste_unscaled)
            print(f"\nMSE (escala original): {mse_unscaled:.4f}")
            print(f"MAE (escala original): {mae_unscaled:.4f} (Erro médio em USD)")

            # Visualização das Previsões vs Valores Reais
            plt.figure(figsize=(14, 5))
            # Usa o índice original do dataframe pré-processado para o eixo X
            index_teste = dados_processamento.index[len(X_treino):]
            plt.plot(index_teste, y_teste_unscaled, label='Valores Reais (Teste)', color='blue', marker='.', linestyle='None', alpha=0.7)
            plt.plot(index_teste, previsoes_teste_unscaled, label='Previsões do Modelo (Teste)', color='red', alpha=0.7)
            plt.title('Previsões do Modelo vs Valores Reais (Conjunto de Teste)')
            plt.xlabel('Índice Original dos Dados')
            plt.ylabel('Preço de Fechamento (USD)')
            plt.legend()
            plt.grid(True)
            plt.show()

        except Exception as e:
            print(f"Erro durante a desnormalização ou plotagem: {e}")
            print("Plotando apenas os dados escalados:")
            # Fallback para plotar dados escalados se a desnormalização falhar
            plt.figure(figsize=(14, 5))
            index_teste = dados_processamento.index[len(X_treino):]
            plt.plot(index_teste, y_teste, label='Valores Reais Scaled (Teste)', color='blue', marker='.', linestyle='None', alpha=0.7)
            plt.plot(index_teste, previsoes_teste_scaled, label='Previsões Scaled (Teste)', color='red', alpha=0.7)
            plt.title('Previsões Scaled vs Reais Scaled (Conjunto de Teste)')
            plt.xlabel('Índice Original dos Dados')
            plt.ylabel('Valor Escalado (0-1)')
            plt.legend()
            plt.grid(True)
            plt.show()
    else:
        print("Não foi possível carregar o scaler. Plot e métricas desnormalizadas foram pulados.")

else:
    print("\nERRO: Modelo não treinado ou dados de teste não disponíveis. Avaliação pulada.")

## 7. API Flask e Previsão

Para tornar o modelo útil, criamos uma API web usando Flask. A API tem rotas para: [cite: main.py, 6, 11]

* `/historico`: Retornar os dados históricos armazenados no banco. [cite: main.py]
* `/treinarmodelo`: Disparar o processo de coleta, armazenamento e treinamento (como fizemos nas etapas anteriores). [cite: main.py]
* `/prever`: Receber (ou coletar internamente) os dados mais recentes, pré-processá-los *usando o mesmo scaler* que foi salvo, fazer a previsão com o modelo treinado e retornar o valor previsto (revertido para a escala original). [cite: main.py]
* `/`: Exibir o dashboard. [cite: main.py]

Abaixo, simulamos a lógica da rota `/prever`.

In [ ]:
print("\n--- Simulação da Lógica de Previsão (como na API /prever) ---")

# 1. Carregar o modelo e scaler salvos localmente
modelo_carregado = None
scaler_carregado = None
modelo_filename = 'modelo.pkl'
scaler_filename = 'scaler.pkl'

try:
    with open(modelo_filename, 'rb') as f:
        modelo_carregado = pickle.load(f)
    print(f"Modelo '{modelo_filename}' carregado.")
except Exception as e:
    print(f"Erro ao carregar modelo '{modelo_filename}': {e}")

try:
    with open(scaler_filename, 'rb') as f:
        scaler_carregado = pickle.load(f)
    print(f"Scaler '{scaler_filename}' carregado.")
except Exception as e:
    print(f"Erro ao carregar scaler '{scaler_filename}': {e}")


# Procede apenas se ambos foram carregados e temos dados de teste
if modelo_carregado and scaler_carregado and 'X_teste' in locals() and not X_teste.empty:

    # 2. Obter os dados mais recentes (simulamos usando a última linha do conjunto de teste)
    # Na API real, buscaríamos com coletar_dados_tempo_real() e aplicaríamos
    # o pré-processamento (rolling, pct_change, dropna, scale)
    dados_recentes_features_scaled = X_teste.iloc[-1:].copy() # Pega a última linha como um DataFrame
    print("\nDados de entrada (simulados da última linha do teste, já escalados):")
    print(dados_recentes_features_scaled)

    # 3. Fazer a previsão com o modelo carregado
    previsao_scaled = modelo_carregado.predict(dados_recentes_features_scaled)[0]
    print(f"\nPrevisão (escalada): {previsao_scaled:.6f}")

    # 4. Reverter a previsão para a escala original
    try:
        # Cria um array com a mesma estrutura das features originais,
        # substituindo o valor da feature 'fechamento' pela previsão escalada.
        input_data_for_inverse = dados_recentes_features_scaled.values[0].copy() # Faz uma cópia para não alterar o original
        target_index = features.index('fechamento') # 'features' foi definido na célula de pré-proc
        input_data_for_inverse[target_index] = previsao_scaled

        # Aplica inverse_transform no array completo
        previsao_unscaled = scaler_carregado.inverse_transform([input_data_for_inverse])[0, target_index]

        print(f"\nPrevisão Final (escala original - USD): ${previsao_unscaled:.2f}")

        # Guarda a previsão para usar no gráfico final
        previsao_final_simulada = previsao_unscaled

    except Exception as e:
        print(f"Erro ao reverter a escala da previsão: {e}")
        previsao_final_simulada = None

else:
    print("\nNão foi possível carregar o modelo e/ou scaler ou não há dados de teste. Simulação da previsão pulada.")
    previsao_final_simulada = None

## 8. Dashboard

Finalmente, para apresentar os resultados de forma visual e acessível, criamos um dashboard simples usando Flask e HTML. [cite: 11, main.py, templates/dashboard.html]

O dashboard exibe:
1.  **Gráfico Histórico:** Um gráfico gerado com Matplotlib mostrando a evolução do preço de fechamento, lido do banco de dados. [cite: main.py]
2.  **Previsão para o Próximo Dia:** O valor previsto pelo modelo através da rota `/prever`. [cite: main.py]

A API Flask gera o gráfico, o converte para base64 e o embute diretamente na página HTML (`dashboard.html`) junto com o valor da previsão. [cite: main.py, templates/dashboard.html]

Abaixo, mostramos como o gráfico histórico é gerado (sem a parte do Flask/HTML).

In [ ]:
print("\n--- Simulação da Geração do Gráfico para o Dashboard ---")

# Usa os dados lidos do banco ou da memória (já em dados_para_analise)
if 'dados_para_analise' in locals() and not dados_para_analise.empty:
    dados_grafico = dados_para_analise.copy()
    # Garante que data_pregao seja datetime
    dados_grafico['data_pregao'] = pd.to_datetime(dados_grafico['data_pregao'])


    # Pega a previsão final simulada na célula anterior
    previsao_amanha_simulada = previsao_final_simulada if 'previsao_final_simulada' in locals() else None

    plt.figure(figsize=(12, 6))
    plt.plot(dados_grafico['data_pregao'], dados_grafico['fechamento'], label='Preço de Fechamento Histórico', color='blue')

    # Adiciona a previsão simulada ao gráfico (se disponível e válida)
    if previsao_amanha_simulada is not None and pd.notna(previsao_amanha_simulada):
        # Data de amanhã (baseada na última data dos dados)
        ultima_data = dados_grafico['data_pregao'].iloc[-1]
        data_previsao = ultima_data + timedelta(days=1)
        plt.scatter(data_previsao, previsao_amanha_simulada, color='red', label=f'Previsão Próximo Dia (${previsao_amanha_simulada:.2f})', zorder=5, s=100)
        print(f"\nAdicionando ponto de previsão para {data_previsao.strftime('%Y-%m-%d')} no valor de ${previsao_amanha_simulada:.2f}")
    else:
        print("\nNenhuma previsão final válida para adicionar ao gráfico.")

    plt.title(f'Histórico de Preços de Fechamento {ticker} com Previsão Simulada')
    plt.xlabel('Data')
    plt.ylabel('Preço (USD)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

else:
    print("\nERRO: Não há dados carregados para gerar o gráfico final.")

## 9. Conclusão

Este projeto demonstrou o fluxo completo de um projeto de Machine Learning aplicado à previsão de preços de ações, cumprindo os requisitos do Tech Challenge: [cite: 6, 7, 8, 11]

1.  **Coleta e Armazenamento:** Dados da GOOG foram coletados via `yfinance` e estruturados para armazenamento em PostgreSQL com SQLAlchemy. [cite: main.py]
2.  **Pré-processamento e Modelagem:** Realizamos engenharia de features, normalização e treinamos um modelo Random Forest Regressor com otimização de hiperparâmetros (`GridSearchCV`). [cite: main.py]
3.  **Avaliação:** O modelo foi avaliado com métricas como MSE e MAE. [cite: main.py]
4.  **Produção:** Uma API Flask foi criada para servir previsões e um dashboard simples para visualização dos resultados. [cite: main.py, 11]
5.  **Documentação:** O código está documentado e disponível no GitHub. [cite: 8, techchallenge3/README.md]

**Próximos Passos Possíveis:**
* Experimentar outros modelos (LSTM, ARIMA).
* Adicionar mais features (sentimento de notícias, indicadores macroeconômicos).
* Melhorar o dashboard com mais interatividade.
* Deploy da solução em nuvem (AWS, GCP, Azure).

Obrigado!